## FINAL PROJECT.

### Client: AtliQ Hardware.

### AtliQ Hardware is one of the leading computer hardware producers in India, and has even expanded into other countries as well. They sell different types of hardware to big players such as Amazon, Best Buy, and Walmart.<br>
### This year, they're asking PWC to conduct a big audit of their sales and help them automate their existing data.

***

### FINNANCIAL ANALYSIS

#### DECOMPOSITION AND RESEARCH TASKS:

### 1. Definition of Objectives and Key Questions

#### Objective: Understand how revenue, profits, and margins have changed over time. Analyze shifts in the market and identify the most profitable categories.<br>
#### Key Questions:<br>
- How have revenue and profits trended over the years?<br>
- What are the profit margins, and how have they evolved?<br>
- Which markets, platforms, or channels have driven the most revenue and profit?<br>
- What categories of products are the most profitable?<br>
- How have different regions/subzones contributed to revenue and profit?<br>
- Are there any noticeable shifts in market trends or customer preferences?

***

### 2. Identify the Important Metrics

#### Revenue Metrics:<br>
- Total Revenue: Sum of gross_price * sold_quantity from fact_sales_monthly.
- Revenue Growth Rate: Percentage increase/decrease in revenue year-over-year.
#### Profit Metrics:
- Gross Profit: Revenue minus the manufacturing cost (gross_price * sold_quantity - manufacturing_cost * sold_quantity).
- Profit Margin: Gross profit as a percentage of total revenue.
- Net Profit: Gross profit minus pre-invoice discounts (gross_profit - (pre_invoice_discount_pct * gross_profit)).
- Net Profit Margin: Net profit as a percentage of total revenue.
#### Market and Category Metrics:
- Revenue by Market: Revenue broken down by market.
- Revenue by Platform: Revenue broken down by platform.
- Profit by Category: Profitability of different segment and category.
#### Trend Metrics:<br>
- Market Share: Proportion of total revenue by market.
- Revenue/Profit by Year: Trend of revenue and profit over time (fiscal_year).

***

### 3. Data Preparation and Cleaning

#### Data Integrity Check:<br>
- Ensure each market belongs to one sub_zone and region.
- Validate that each customer has only one associated platform.
- Check that product_code is consistent across all fact tables.
- Verify that there are only 3 unique values in the division column of the dim_product table.
#### Handle Missing Data:<br>
- Identify and address missing values in key columns like gross_price, manufacturing_cost, and sold_quantity.
- Impute or remove records with missing data as appropriate.
#### Feature Engineering:<br>
- Create new features like gross_profit, net_profit, profit_margin, and net_profit_margin for further analysis.
- Aggregate data by fiscal_year, market, platform, etc., to facilitate trend analysis.

***

### 4. Exploratory Data Analysis (EDA)

#### Descriptive Statistics:<br>
- Calculate mean, median, and standard deviation for revenue, profit, and margin metrics.
- Identify outliers or unusual trends in the data.
#### Trend Analysis:<br>
- Plot revenue, profit, and margin over time to visualize trends.
- Use time series analysis to forecast future revenue and profit.
#### Segment Analysis:<br>
- Break down revenue and profit by platform, market, region, and category.
- Identify which segments contribute most to overall profitability.
#### Correlation Analysis:<br>
- Analyze the relationship between different metrics (e.g., how does discount percentage affect net profit?).
- Look for correlations between market conditions and profitability.

***

### 5. Identifying Problems or Questions to Address

#### Profitability Issues:<br>
- Are certain markets or platforms less profitable? If so, why?
- Is there a declining trend in any key categories or segments?
#### Market Shifts:<br>
- Are there shifts in revenue from one platform to another (e.g., from Brick & Mortar to E-Commerce)?
- Are there emerging markets or declining markets based on the data?
#### Cost Efficiency:<br>
- Are manufacturing costs rising, and how does this affect profit margins?
- Are discounts eroding profitability in certain markets or for certain customers?

***

### 6. Research and Contextual Understanding

#### Industry Benchmarks:<br>
- Research industry standards for profit margins, cost structures, and market share to compare with your findings.
#### Economic Factors:<br>
- Consider external economic factors that may have influenced market trends (e.g., currency fluctuations, economic downturns).
#### Competitor Analysis:
- Research competitors' performance in similar markets to understand potential threats or opportunities.


***

### 7. Hypothesis Formation and Testing

#### Based on the EDA, form hypotheses about what drives revenue and profit. For example:<br>
- "E-Commerce platform is more profitable due to lower overhead costs."
- "The market in Japan has seen a decline due to increased competition."
#### Design experiments or statistical tests to validate these hypotheses.

***

### 8. Visualization and Reporting

#### Dashboards:<br>
- Create visualizations to represent the trends, comparisons, and insights derived from the data.
- Use tools like Power BI, Tableau, or Python libraries like Matplotlib/Seaborn.
#### Report:<br>
- Compile a report summarizing your findings, insights, and recommendations based on the analysis.
- Ensure that the report answers the key questions posed at the beginning of the project.

***

### 9. Actionable Recommendations

#### Based on the analysis, provide recommendations for business strategies to improve revenue, optimize profits, and adapt to market shifts. These could include:<br>
- Focusing more on profitable platforms or markets.
- Reducing costs in certain product lines.
- Increasing marketing efforts in emerging markets or declining segments.

In [22]:
import sqlite3
import pandas as pd

In [23]:
con = sqlite3.connect('/Users/cesarchaparro/Desktop/TripleTen/Sprint_12/Final_Project/atliq_db.sqlite3')

In [24]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('dim_customer',), ('dim_product',), ('fact_pre_discount',), ('fact_manufacturing_cost',), ('fact_gross_price',), ('fact_sales_monthly',)]


#### OPEN EACH TABLE INDIVIDUALLY

In [25]:
query = """Select * from 
dim_customer
LIMIT 10
"""
dim_customer = pd.read_sql_query(query, con)
dim_customer.head()

,customer_code,customer,platform,channel,market,sub_zone,region
0,70002017,Atliq Exclusive,Brick & Mortar,Direct,India,India,APAC
1,70002018,Atliq e Store,E-Commerce,Direct,India,India,APAC
2,70003181,Atliq Exclusive,Brick & Mortar,Direct,Indonesia,ROA,APAC
3,70003182,Atliq e Store,E-Commerce,Direct,Indonesia,ROA,APAC
4,70004069,Atliq Exclusive,Brick & Mortar,Direct,Japan,ROA,APAC


In [26]:
query = """SELECT customer, COUNT(DISTINCT platform) AS platform_count
FROM dim_customer
GROUP BY customer
HAVING COUNT(DISTINCT platform) > 1;
"""
result = pd.read_sql_query(query, con)
print(result)

Empty DataFrame
Columns: [customer, platform_count]
Index: []


##### There are no customer who have more than one platform. Each customer has one platform.

In [27]:
query = """SELECT market, 
       COUNT(DISTINCT sub_zone) AS distinct_subzones, 
       COUNT(DISTINCT region) AS distinct_regions
FROM dim_customer
GROUP BY market
HAVING distinct_subzones > 1 OR distinct_regions > 1;
"""
result = pd.read_sql_query(query, con)
print(result)


Empty DataFrame
Columns: [market, distinct_subzones, distinct_regions]
Index: []


##### Each market has been checked as having only 1 subzone and 1 region under it.

***

In [28]:
query = """Select * from 
dim_product
LIMIT 10
"""
dim_product = pd.read_sql_query(query, con)
dim_product.head()

,product_code,division,segment,category,product,variant
0,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard
1,A0118150102,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Plus
2,A0118150103,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Premium
3,A0118150104,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Premium Plus
4,A0219150201,P & A,Peripherals,Internal HDD,AQ WereWolf NAS Internal Hard Drive HDD – 8.89 cm,Standard


In [33]:
query = """SELECT COUNT(DISTINCT division) as unique_divisions 
FROM dim_product;
"""
result = pd.read_sql_query(query, con)
print(result)

if result['unique_divisions'][0] == 3:
    print("There are exactly 3 unique values in the 'division' column.")
else:
    print(f"There are {result['unique_divisions'][0]} unique values in the 'division' column.")

   unique_divisions
0                 3
There are exactly 3 unique values in the 'division' column.


##### After verifying the column I can conclude that there are only 3 types of divisions on the column.

In [35]:
query = """SELECT COUNT(DISTINCT segment) as unique_segments 
FROM dim_product;
"""
result = pd.read_sql_query(query, con)
print(result)

if result['unique_segments'][0] == 6:
    print("There are exactly 6 unique values in the 'segment' column.")
else:
    print(f"There are {result['unique_segments'][0]} unique values in the 'segment' column.")

   unique_segments
0                6
There are exactly 6 unique values in the 'segment' column.


##### After verifying the segments column I can conclude that there are only 6 types of segments on the column.

***

In [29]:
query = """Select * from 
fact_pre_discount
LIMIT 10
"""
fact_pre_discount = pd.read_sql_query(query, con)
fact_pre_discount.head()

,customer_code,fiscal_year,pre_invoice_discount_pct
0,70002017,2018,0.0824
1,70002017,2019,0.0777
2,70002017,2020,0.0735
3,70002017,2021,0.0703
4,70002017,2022,0.1057


In [30]:
query = """Select * from 
fact_manufacturing_cost
LIMIT 10
"""
fact_manufacturing_cost = pd.read_sql_query(query, con)
fact_manufacturing_cost.head()

,product_code,cost_year,manufacturing_cost
0,A0118150101,2018,4.6190
1,A0118150101,2019,4.2033
2,A0118150101,2020,5.0207
3,A0118150101,2021,5.5172
4,A0118150102,2018,5.6036


In [31]:
query = """Select * from 
fact_gross_price
LIMIT 10
"""
fact_gross_price = pd.read_sql_query(query, con)
fact_gross_price.head()

,product_code,fiscal_year,gross_price
0,A0118150101,2018,15.3952
1,A0118150101,2019,14.4392
2,A0118150101,2020,16.2323
3,A0118150101,2021,19.0573
4,A0118150102,2018,19.5875


In [32]:
query = """Select * from 
fact_sales_monthly
LIMIT 10
"""
fact_sales_monthly = pd.read_sql_query(query, con)
fact_sales_monthly.head()

,date,product_code,customer_code,sold_quantity,fiscal_year
0,2017-09-01,A0118150101,70002017,51,2018
1,2017-09-01,A0118150101,70002018,77,2018
2,2017-09-01,A0118150101,70003181,17,2018
3,2017-09-01,A0118150101,70003182,6,2018
4,2017-09-01,A0118150101,70006157,5,2018
